In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.1 MB/s eta 0:00:00


In [2]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 10.4 MB/s eta 0:00:00


In [3]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
from google.colab import userdata


In [6]:
from typing import Tuple, List, Optional

In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [8]:
from langchain_core.runnables import ConfigurableField

In [9]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [10]:
import os

In [11]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [12]:
from langchain_community.vectorstores import Neo4jVector

In [14]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [15]:
from langchain_community.graphs import Neo4jGraph

In [16]:
graph = Neo4jGraph()

<ipython-input-16-6e7385464d00>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [17]:
import pandas as pd

df = pd.read_csv("/content/invo_10.csv", low_memory=False)




In [18]:

from langchain_core.documents import Document
from langchain.text_splitter import TokenTextSplitter



# Convert each row to a single string and wrap in Document
documents = []
for i, row in df.iterrows():
    row_text = " | ".join([f"{col}: {row[col]}" for col in df.columns])
    documents.append(Document(page_content=row_text))

# Split documents into smaller chunks
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
chunked_documents = text_splitter.split_documents(documents)


In [19]:
chunked_documents

[Document(metadata={}, page_content='Date: 04-10-2024 | Timestamp: 2024-10-04 18:53:25 UTC | Invoice_No_: C1984 | Payment_Type: Cash | Order_Type: Dine In | Area: Garden Table | Item_Name: Exotic Stir Fried Vegetable | Price: 795 | Qty_: 1.0 | Sub_Total: 795.0 | Discount: 0.0 | Tax: 79.6 | Final_Total: 874.6 | Status: Complimentary | Table_No_: G 5 | Server_Name: TAPASH | Covers: 3 | Variation: nan | Category: ORIENTAL STARTER | HSN: nan | Phone: 9843950806 | Name: PHOTO SHOOT | Address: nan | GST: nan | Assign_To: nan | Non_Taxable: 0.0 | CGST_Rate: 2.5 | CGST_Amount: 19.9 | SGST_Rate: 2.5 | SGST_Amount: 19.9 | VAT_Rate: 10 | VAT_Amount: 0.0 | Service_Charge_Rate: 5 | Service_Charge_Amount: 39.8 | Date_z: nan | Invoice_Date_z: nan | Aggregator_Order_No_z: nan | PoS_Invoice_No_z: nan | Order_From_z: nan | Outlet_Name_z: nan | Petpooja_Identifier_z: nan | Order_Type_z: nan | Customer_Name_z: nan | Customer_Phone_z: 9843950806 | Payment_Type_z: nan | Delivery_status_z: nan | Status_z: na

In [20]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [21]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [22]:
graph_documents = llm_transformer.convert_to_graph_documents(chunked_documents)

In [23]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [24]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [25]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [26]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [27]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [28]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [29]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [30]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [31]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# Extract entities from sales data text
class Entities(BaseModel):
    """Key extracted entities from restaurant sales records."""

    names: List[str] = Field(
        ...,
        description="List of extracted entities such as server names, customer names, item names, or business-related references present in the record.",
    )


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an intelligent assistant that extracts structured entities from restaurant sales records. "
            "Focus on extracting relevant entities such as server names, customer names, item names, payment types, and any anomaly indicators.",
        ),
        (
            "human",
            "Extract entities using the specified schema from the following input:\n\n{question}",
        ),
    ]
)


In [33]:
entity_chain = prompt | llm.with_structured_output(Entities)

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [34]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [35]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [36]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [37]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [38]:
def retriever(question: str):
    print(f"Search query: {question}")

    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]

    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ".join(unstructured_data)}
    """
    return final_data


In [39]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [40]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [41]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [42]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)


In [43]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language.
Answer:"""

In [44]:
prompt = ChatPromptTemplate.from_template(template)

In [45]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# print(book_appointment("Dr. Alok Sardesai", "John Doe", "2025-03-22", "10:00 AM"))

In [ ]:
# print(check_availability("Dr. Alok Sardesai", "2025-03-22"))

In [46]:
chain.invoke({"question": "who is the most busy server?"})

Search query: who is the most busy server?


<ipython-input-35-bcc56ddd5940>:3: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]


'Based on the provided data, the most busy server appears to be TAPASH, as they are mentioned in multiple transactions. Specifically, TAPASH served during the orders for "PANEER CHILLY DRY" and "Burnt Garlic Soup (VEG)." In contrast, TANMOY served only one order for "Manchow Soup (VEG)." Therefore, TAPASH is the busiest server among those listed.'